In [6]:
# Production notebook: load saved preds -> SLSQP prob-space blend -> single temp scaling -> write submission
import sys, subprocess, time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

def ensure_scipy():
    try:
        import scipy  # noqa: F401
        from scipy.optimize import minimize  # noqa: F401
        return True
    except Exception:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'scipy'], check=True)
        return True

def clip_norm(P):
    P = np.clip(P, 1e-15, 1-1e-15)
    return P / P.sum(axis=1, keepdims=True)

# Load data and labels (for OOF-based blending calibration)
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
id_col = 'id'; target_col = 'species'
feature_cols = [c for c in train.columns if c not in [id_col, target_col]]
le = LabelEncoder()
y_enc = le.fit_transform(train[target_col].values)
K = len(le.classes_)

# Load candidate OOF/test predictions (lr_ens replaces lr_base)
cands = []; names = []
oof_lr = np.load('oof_lr_ens.npy'); test_lr = np.load('test_lr_ens.npy')
cands.append((oof_lr, test_lr)); names.append('lr_ens')
try:
    oof_concat = np.load('oof_concat_lr.npy'); test_concat = np.load('test_concat_lr.npy')
    cands.append((oof_concat, test_concat)); names.append('concat_lr')
except Exception as e:
    print('Missing concat_lr, continuing:', e)
try:
    oof_c2 = np.load('oof_chi2_lr.npy'); test_c2 = np.load('test_chi2_lr.npy')
    cands.append((oof_c2, test_c2)); names.append('chi2_lr')
except Exception as e:
    print('Missing chi2_lr, continuing:', e)
try:
    oof_blk = np.load('oof_block_hell_lr.npy'); test_blk = np.load('test_block_hell_lr.npy')
    cands.append((oof_blk, test_blk)); names.append('blockL1_lr')
except Exception as e:
    print('Missing blockL1_lr, continuing:', e)
try:
    oof_hc = np.load('oof_hell_center_only.npy'); test_hc = np.load('test_hell_center_only.npy')
    cands.append((oof_hc, test_hc)); names.append('hell_center_only')
except Exception as e:
    print('Missing hell_center_only, continuing:', e)
try:
    oof_vA = np.load('oof_hell_varA_noGL2.npy'); test_vA = np.load('test_hell_varA_noGL2.npy')
    cands.append((oof_vA, test_vA)); names.append('hell_varA_noGL2')
except Exception as e:
    print('Missing hell_varA_noGL2, continuing:', e)
try:
    oof_rp = np.load('oof_ridge_platt.npy'); test_rp = np.load('test_ridge_platt.npy')
    cands.append((oof_rp, test_rp)); names.append('ridge_platt')
except Exception as e:
    print('Missing ridge_platt, continuing:', e)
try:
    oof_lda = np.load('oof_lda_lsqr.npy'); test_lda = np.load('test_lda_lsqr.npy')
    cands.append((oof_lda, test_lda)); names.append('lda_lsqr')
except Exception as e:
    print('Missing lda_lsqr, continuing:', e)
try:
    oof_xgb = np.load('oof_xgb_gblinear.npy'); test_xgb = np.load('test_xgb_gblinear.npy')
    cands.append((oof_xgb, test_xgb)); names.append('xgb_gblinear')
except Exception as e:
    print('Missing xgb_gblinear, continuing:', e)

assert len(cands) >= 2, 'Need at least two models to blend'
oofs = [o for o,_ in cands]
tests = [t for _,t in cands]

ensure_scipy()
from scipy.optimize import minimize

def blend_from_w(w, mats):
    P = np.zeros_like(mats[0])
    for wi, Pi in zip(w, mats):
        P += wi * Pi
    return clip_norm(P)

def objective(w):
    P = blend_from_w(w, oofs)
    return log_loss(y_enc, P, labels=list(range(K)))

m = len(cands)
w0 = np.ones(m, dtype=np.float64) / m
bounds = [(0.0, 1.0)] * m
cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},)
t0 = time.time()
res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=cons, options={'maxiter': 800, 'ftol': 1e-10, 'disp': False})
w_opt = res.x if res.success else w0
w_opt = np.maximum(w_opt, 0); w_opt = w_opt / w_opt.sum()
oof_blend = blend_from_w(w_opt, oofs)
test_blend = blend_from_w(w_opt, tests)
oof_ll = log_loss(y_enc, oof_blend, labels=list(range(K)))
print('SLSQP success:', res.success, '| OOF(pre-cal):', round(oof_ll, 6), '| Weights:', dict(zip(names, np.round(w_opt, 4))), '| time {:.2f}s'.format(time.time()-t0))

# Single global temperature scaling
def temp_scale(P, a):
    Ps = np.power(np.clip(P, 1e-15, 1-1e-15), float(a))
    return Ps / Ps.sum(axis=1, keepdims=True)

alphas = np.linspace(0.6, 2.4, 181)
best = (1.0, 1e9)
for a in alphas:
    ll = log_loss(y_enc, temp_scale(oof_blend, a), labels=list(range(K)))
    if ll < best[1]:
        best = (float(a), float(ll))
print(f'Temp scaling on blend: alpha={best[0]:.4f}, OOF={best[1]:.6f}')
test_cal = temp_scale(test_blend, best[0])

# Write soft submission
pred_df = pd.DataFrame(test_cal, columns=list(le.classes_))
sub_cols = [c for c in sample.columns if c != id_col]
pred_df = pred_df[sub_cols]
submission = pd.concat([test[[id_col]].reset_index(drop=True), pred_df.reset_index(drop=True)], axis=1)
submission.to_csv('submission.csv', index=False)
print('Saved submission.csv:', submission.shape)

SLSQP success: True | OOF(pre-cal): 0.034656 | Weights: {'lr_ens': 0.2403, 'concat_lr': 0.0, 'chi2_lr': 0.0, 'blockL1_lr': 0.0, 'hell_center_only': 0.0681, 'hell_varA_noGL2': 0.6916, 'ridge_platt': 0.0, 'lda_lsqr': 0.0, 'xgb_gblinear': 0.0} | time 1.02s


Temp scaling on blend: alpha=1.6700, OOF=0.025725
Saved submission.csv: (99, 100)


In [5]:
# Experimental: logit-space SLSQP blend + single global temperature; writes alt submission
import numpy as np, pandas as pd, time
from sklearn.metrics import log_loss
from scipy.optimize import minimize

def clip_norm(P):
    P = np.clip(P, 1e-15, 1-1e-15)
    return P / P.sum(axis=1, keepdims=True)

# Reuse oofs/tests, y_enc, names from previous cell's scope
oofs = [o for o,_ in cands]
tests = [t for _,t in cands]
K = len(le.classes_)

def to_logits(P):
    return np.log(np.clip(P, 1e-15, 1.0))

oofs_log = [to_logits(P) for P in oofs]
tests_log = [to_logits(P) for P in tests]

def softmax_rows(L):
    Lm = L - L.max(axis=1, keepdims=True)
    P = np.exp(Lm)
    return P / P.sum(axis=1, keepdims=True)

def blend_logits(w, mats_log):
    L = np.zeros_like(mats_log[0])
    for wi, Li in zip(w, mats_log):
        L += wi * Li
    return L

def objective_logit(w):
    L = blend_logits(w, oofs_log)
    P = softmax_rows(L)
    return log_loss(y_enc, P, labels=list(range(K)))

m = len(oofs_log)
w0 = np.ones(m, dtype=np.float64) / m
bounds = [(0.0, 1.0)] * m
cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},)
t0 = time.time()
res = minimize(objective_logit, w0, method='SLSQP', bounds=bounds, constraints=cons, options={'maxiter': 800, 'ftol': 1e-10, 'disp': False})
w_opt = res.x if res.success else w0
w_opt = np.maximum(w_opt, 0); w_opt = w_opt / w_opt.sum()
L_oof = blend_logits(w_opt, oofs_log)
L_test = blend_logits(w_opt, tests_log)
P_oof = softmax_rows(L_oof)
P_test = softmax_rows(L_test)
oof_ll = log_loss(y_enc, P_oof, labels=list(range(K)))
print('Logit-SLSQP success:', res.success, '| OOF(pre-cal):', round(oof_ll, 6), '| Weights:', dict(zip(names, np.round(w_opt, 4))), '| time {:.2f}s'.format(time.time()-t0))

# Global temperature on logits: L / T
Ts = np.linspace(0.4, 2.5, 212)
best = (1.0, 1e9)
for T in Ts:
    Po = softmax_rows(L_oof / float(T))
    ll = log_loss(y_enc, Po, labels=list(range(K)))
    if ll < best[1]:
        best = (float(T), float(ll))
print(f'Logit temp scaling: T={best[0]:.4f}, OOF={best[1]:.6f}')
P_test_cal = softmax_rows(L_test / best[0])

# Write alternate submission
pred_df = pd.DataFrame(P_test_cal, columns=list(le.classes_))
sub_cols = [c for c in sample.columns if c != id_col]
pred_df = pred_df[sub_cols]
submission_alt = pd.concat([test[[id_col]].reset_index(drop=True), pred_df.reset_index(drop=True)], axis=1)
submission_alt.to_csv('submission_logit_blend_calibrated.csv', index=False)
print('Saved submission_logit_blend_calibrated.csv:', submission_alt.shape)

Logit-SLSQP success: True | OOF(pre-cal): 0.031086 | Weights: {'lr_ens': 0.2039, 'concat_lr': 0.2132, 'chi2_lr': 0.1686, 'blockL1_lr': 0.0569, 'hell_center_only': 0.3573} | time 0.49s


Logit temp scaling: T=0.6787, OOF=0.026404
Saved submission_logit_blend_calibrated.csv: (99, 100)


In [ ]:
# Overwrite submission.csv with best (logit-blend calibrated) predictions
import pandas as pd
best_sub_path = 'submission_logit_blend_calibrated.csv'
sub_df = pd.read_csv(best_sub_path)
sub_df.to_csv('submission.csv', index=False)
print('submission.csv overwritten from', best_sub_path, '| shape:', sub_df.shape)